# **Google Vision Product Search:** 
## Build a Product catalog recognition engine in one hour

Vision API Product Search allows retailers to create products, each containing reference images that visually describe the product from a set of viewpoints. Retailers can then add these products to product sets. Currently Vision API Product Search supports the following product categories: homegoods, apparel, toys, packaged goods, and general .

When users query the product set with their own images, Vision API Product Search applies machine learning to compare the product in the user's query image with the images in the retailer's product set, and then returns a ranked list of visually and semantically similar results.

After loading your catalog into Vision Product Search, you'll be able to search for similar products in your catalog by providing a image

> This notebook is using a Kaggle dataset for product recognition. The goal is to extract a csv for bulk import in Product Vision Search  

**Useful links:**  
https://github.com/zinjiggle/google-product-search-simple-ui  
https://github.com/GoogleCloudPlatform/python-docs-samples/tree/master/vision/cloud-client/product_search

## 0. Install vision library

In [ ]:
#pip install google-cloud-vision

In [2]:
#pip install --upgrade google-cloud-storage

In [5]:
project_id='pod-fr-retail'
location='europe-west1'
product_set='kaggle_shoes'
bucket_name="pod-fr-retail-kaggle"
gcs_bucket="gs://"+bucket_name+"/"

## 1. Create a Products catalog with Bigquery
The first step shows how to prepare a product catalog for **Vision Product Search** in BigQuery:
* **image-uri**: The Google Cloud Storage URI of the reference image.
* **image-id**: Optional. A unique value if you supply it. Otherwise, the system will assign a unique value.
* **product-set-id**: A unique identifier for the product set to import the images into.
* **product-id**: A user-defined ID for the product identified by the reference image. A product-id can be associated with multiple reference images. Note: A single product may also belong to several product sets. If a product-id already exists on bulk import then product-category, product-display, and labels are ignored for that line entry.
* **product-category**: Allowed values are homegoods-v2, apparel-v2, toys-v2, packagedgoods-v1, and general-v1 *; the category for the product identified by the reference image. Inferred by the system if not specified in the create request. Allowed values are also listed in the productCategory reference documentation. Legacy productCategory codes: Legacy categories (homegoods, apparel, and toys) are still supported, but the updated -v2 categories should be used for new products.
* **product-display-name**: Optional. If you don't provide a name for the product displayName will be set to " ". You can update this value later.
* **labels**: Optional. A string (with quotation marks) of key-value pairs that describe the products in the reference image. For example:"color=black,style=formal"
* **bounding-poly**: Optional. Specifies the area of interest in the reference image. If a bounding box is not specified: Bounding boxes for the image are inferred by the Vision API; multiple regions in a single image may be indexed if multiple products are detected by the API. The line must end with a comma. See the example below for a product without a bounding poly specified.If you include a bounding box, the boundingPoly column should contain an even number of comma-separated numbers, with the format p1_x,p1_y,p2_x,p2_y,...,pn_x,pn_y. An example line looks like this: 0.1,0.1,0.9,0.1,0.9,0.9,0.1,0.9.

#### 1.1 Product Catalog exploration (Kaggle dataset) 

Let's start by exploring our data. We are using a product images catalog from a kaggle contest. Goal was to classify automatically product based on a image. The dataset contains 48 products main categories and around 12 millions images. For our notebook quickstart, we'll focus on Shoes category.

In [2]:
%%bigquery
SELECT category_level1, count(*) num_products  
FROM 
    `pod-fr-retail.kaggle.train_images` a 
JOIN `pod-fr-retail.kaggle.category_names`  b 
ON CAST(b.category_id AS STRING) =(REGEXP_EXTRACT(a.path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/([^-]*)'))
GROUP BY 1 ORDER BY 2 desc
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.96rows/s]


,category_level1,num_products
0,TELEPHONIE - GPS,1227001
1,AUTO - MOTO,1193619
2,INFORMATIQUE,1124907
3,DECO - LINGE - LUMINAIRE,1111509
4,LIBRAIRIE,863965
5,BIJOUX - LUNETTES - MONTRES,688243
6,BRICOLAGE - OUTILLAGE - QUINCAILLERIE,620366
7,JEUX - JOUETS,551408
8,SPORT,434791
9,BAGAGERIE,434675


In shoes category

In [12]:
%%bigquery
SELECT category_level2
from `pod-fr-retail.kaggle.category_names` 
WHERE category_level1 like 'CHAUSSURES%'
GROUP BY 1

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.26rows/s]


,category_level2
0,BOTTES - BOTTINES
1,CHAUSSURES DETENTE
2,BASKET - SPORTSWEAR
3,CHAUSSURES DE VILLE
4,ACCESSOIRES CHAUSSURES


#### 1.2 Create a table with the appropriate schema from Product Catalog (Kaggle dataset) 

In [94]:
%%bigquery
CREATE OR REPLACE TABLE `pod-fr-retail.kaggle.products_vision_search` AS
SELECT 
    a.* EXCEPT (category_id)
    ,CONCAT(replace(lower(CONCAT('','cl1=',b.category_level1,',cl2=',b.category_level2,',cl3=',b.category_level3,' ')),' ','')) labels
    ,null as poly
FROM (
    SELECT 
        path_to_images image_uri
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/([0-9]*-[0-9]*-[0-9]*)')) AS image_id
        ,'kaggle_shoes' as product_set_id
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/[0-9]*-([0-9]*)')) AS product_id
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/([^-]*)')) AS category_id
        ,'apparel-v2' product_category
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/[0-9]*-([0-9]*)')) AS product_display_name
    FROM `pod-fr-retail.kaggle.train_images`
  ) a 
JOIN (SELECT * FROM `pod-fr-retail.kaggle.category_names` 
      WHERE 
      #category_level1='HYGIENE - BEAUTE - PARFUM'
      #category_level1='CHAUSSURES - ACCESSOIRES'
      category_level1 like 'CHAUSSURES%'
  ) b 
ON CAST(b.category_id AS STRING) =a.category_id
#WHERE rand()<0.10;

Query complete after 0.00s: 100%|██████████| 5/5 [00:00<00:00, 2846.30query/s]                        


""


In [95]:
%%bigquery
select count(*) number_of_products from `pod-fr-retail.kaggle.products_vision_search`

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.50s/rows]


,number_of_products
0,369325


# 2. Prepare bulk files to create a new set Data extraction (export CSV for bulk import)
Extract from BigQuery to Google Cloud Storage and split file in multiple files (20k lines max per file)

In [62]:
productsTable='products_vision_search'
productsDataset="kaggle"
destinationSplittedCSV="gs://pod-fr-retail-kaggle/products_vision_search/split/part"

In [100]:
%%bigquery
SELECT * FROM `pod-fr-retail.kaggle.products_vision_search` LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.77rows/s]


,image_uri,image_id,product_set_id,product_id,product_category,product_display_name,labels,poly
0,gs://pod-fr-retail-kaggle/train-images/1000000...,1000010547-2980286-2,kaggle_shoes,2980286,apparel-v2,2980286,"cl1=chaussures-accessoires,cl2=bottes-bottines...",NaN
1,gs://pod-fr-retail-kaggle/train-images/2600000...,1000010547-8733036-1,kaggle_shoes,8733036,apparel-v2,8733036,"cl1=chaussures-accessoires,cl2=bottes-bottines...",NaN
2,gs://pod-fr-retail-kaggle/train-images/5700000...,1000010547-18954499-1,kaggle_shoes,18954499,apparel-v2,18954499,"cl1=chaussures-accessoires,cl2=bottes-bottines...",NaN
3,gs://pod-fr-retail-kaggle/train-images/1400000...,1000010547-4475858-2,kaggle_shoes,4475858,apparel-v2,4475858,"cl1=chaussures-accessoires,cl2=bottes-bottines...",NaN
4,gs://pod-fr-retail-kaggle/train-images/1400000...,1000010547-4459554-2,kaggle_shoes,4459554,apparel-v2,4459554,"cl1=chaussures-accessoires,cl2=bottes-bottines...",NaN


In [2]:
from google.cloud import bigquery_storage_v1
project_id='pod-fr-retail'
client_bq = bigquery_storage_v1.BigQueryReadClient()
table = "projects/{}/datasets/{}/tables/{}".format( "pod-fr-retail", "kaggle", "products_vision_search")
requested_session = bigquery_storage_v1.types.ReadSession()
requested_session.table = table
requested_session.data_format = bigquery_storage_v1.types.DataFormat.AVRO
selected_fields=["image_uri","image_id"
                 ,"product_set_id","product_id"
                 ,"product_category","product_display_name"
                 ,"labels","poly"
                 ,"row_num"]
requested_session.read_options.selected_fields=selected_fields
#requested_session.read_options.row_restriction = 'state = "WA"'
modifiers = None
parent = "projects/{}".format(project_id)
session = client_bq.create_read_session(
    parent=parent,
    read_session=requested_session,
    max_stream_count=1,
)

readers=[]
#print(session)
for stream in session.streams:
    readers.append({'session':session,"streamName":stream.name})

In [92]:
import json
import csv
reader = client_bq.read_rows(readers[0]['streamName'])
rows = reader.rows(readers[0]['session'])
i=0
previous_row_num=0
for row in rows:
    if i%20000==0:
        if f:
            f.close()
        f = open("files/demofile-"+str(i)+".txt", "a")
        w = csv.writer(f)
        print(i/20000)
    i=i+1
    w.writerow(row.values())

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0


In [ ]:
%%bash -s $productsTable $destinationCSV
bq extract --print_header=false $1 $2

In [62]:
%%bash -s $destinationCSV
gsutil cat  $1 | sed -n '1,1p'  

gs://pod-fr-retail-kaggle/train-images/4000000/1000010547-13111765-1.jpg,1000010547-13111765-1,kaggle_shoes,13111765,apparel-v2,13111765,"cl1=chaussures-accessoires,cl2=bottes-bottines,cl3=botte",


In [ ]:
%%bash -s $destinationCSV $destinationSplittedCSV
TOTAL_ROWS=$(gsutil cat $1 | wc -l);
MAX_ROWS=20000;
j=1
k=1
for (( i=1; i<=$TOTAL_ROWS; i++ ))
do 
    if [ $(($i%MAX_ROWS)) == 0 ]; then 
        gsutil -q cat $1 | sed -n "$j"','"$i"'p'  |  gsutil -q cp - $2-$k.csv
        echo "File $2-$k.csv created" 
        j=$((i+1))
        k=$((k+1))
    fi
done
if [ $(($i < TOTAL_ROWS)) ]; then 
        gsutil -q cat $1 | sed -n "$j"','"$TOTAL_ROWS"'p'  |  gsutil -q cp - $2-$k.csv
fi

# 3. Product set creation
Bulk import every files to Vision Product Search (to a specific product set id specified in the file itself)

## 3.1 List all files

In [9]:
from google.cloud import storage
client = storage.Client()
bucket_name = "pod-fr-retail-kaggle"
bucket = client.get_bucket(bucket_name)
blobs = client.list_blobs(bucket_name, prefix='products_vision_search/split/')
gcs_uris=[]
for item in blobs:
    gcs_uris.append(item.name)
    print(item.name)

products_vision_search/split/part-1.csv
products_vision_search/split/part-10.csv
products_vision_search/split/part-11.csv
products_vision_search/split/part-12.csv
products_vision_search/split/part-13.csv
products_vision_search/split/part-14.csv
products_vision_search/split/part-15.csv
products_vision_search/split/part-16.csv
products_vision_search/split/part-17.csv
products_vision_search/split/part-18.csv
products_vision_search/split/part-19.csv
products_vision_search/split/part-2.csv
products_vision_search/split/part-3.csv
products_vision_search/split/part-4.csv
products_vision_search/split/part-5.csv
products_vision_search/split/part-6.csv
products_vision_search/split/part-7.csv
products_vision_search/split/part-8.csv
products_vision_search/split/part-9.csv


## 3.2 Import files and store responses (operation) 

In [4]:
from google.cloud import vision_v1

In [38]:
responses=[]
client = vision_v1.ProductSearchClient()
parent = client.location_path(project_id, location)

def callback(operation_future):
        # Handle result.
        result = operation_future.result()
        
for gcs_uri in gcs_uris:
    gcs_source = vision_v1.types.ImportProductSetsGcsSource(csv_file_uri = gcs_bucket+gcs_uri)
    input_config = vision_v1.types.ImportProductSetsInputConfig(gcs_source=gcs_source)
    response = client.import_product_sets(parent, input_config)
    response.add_done_callback(callback)
    responses.append(response)

In [ ]:
for response in responses:
    print(response.operation.name, ':')
    print(response.metadata)
    print('--------------------')

## 3.3 Check index time of product sets 

In [6]:
import time
from google.cloud import vision_v1
client = vision_v1.ProductSearchClient()
parent = client.location_path(project_id, location)
name = client.product_set_path(project_id, location, product_set)
for element in client.list_product_sets(parent):
    print(element)
    print('Last index:',time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(element.index_time.seconds)))
    print('------')
    pass
#print(client.list_products_in_product_set(name))


name: "projects/pod-fr-retail/locations/europe-west1/productSets/kaggle_shoes"
display_name: " "
index_time {
  seconds: 1666077661
  nanos: 500335979
}
index_error {
}

Last index: 2022-10-18 07:21:01
------
name: "projects/pod-fr-retail/locations/europe-west1/productSets/pvs_shoes"
display_name: " "
index_time {
  seconds: 1666077661
  nanos: 500335979
}
index_error {
}

Last index: 2022-10-18 07:21:01
------


# 4. Delete product sets (disabled / do not remove products) 